In [2]:
#new chrome driver

from selenium import webdriver

def OpenChromeDriver():
    #開啟瀏覽器新分頁，下載google chrome driver
    ban_notify=webdriver.ChromeOptions()
    prefs={"profile.default_content_setting_values.notifications" : 2}
    ban_notify.add_experimental_option("prefs",prefs)
    gchrome=webdriver.Chrome(options=ban_notify)
    
    return gchrome

In [3]:
#login

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

import sys
import time
import getpass

def loginfb(driver):
    driver.get("https://www.facebook.com")
    
    username="***"
    pwd="***"
    
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="email"]')))
    tempentering=driver.find_element_by_id("email")
    tempentering.send_keys(username)

    tempentering=driver.find_element_by_id("pass")
    tempentering.send_keys(pwd)

    tempentering.send_keys(Keys.RETURN)
    time.sleep(5)

In [5]:
#抓取影片留言

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from datetime import date
from bs4 import BeautifulSoup
import sys
import time
import pandas as pd


def getVideoComment(url,driver,this_post_date):
    driver.get(url)
    time.sleep(2)
    driver.execute_script("window.scrollTo(0,400)")
    
    comment_btn=driver.find_element_by_css_selector("div.oajrlxb2.gs1a9yip.g5ia77u1.mtkw9kbi.tlpljxtp.qensuy8j.ppp5ayq2.goun2846.ccm00jje.s44p3ltw.mk2mc5f4.rt8b4zig.n8ej3o3l.agehan2d.sk4xxmp2.rq0escxv.nhd2j8a9.pq6dq46d.mg4g778l.btwxx1t3.pfnyh3mw.p7hjln8o.kvgmc6g5.cxmmr5t8.oygrvhab.hcukyx3x.tgvbjcpo.hpfvmrgz.jb3vyjys.rz4wbd8a.qt6c0cv9.a8nywdso.l9j0dhe7.i1ao9s8h.esuyzwwr.f1sip0of.du4w35lb.lzcic4wl.abiwlrkh.gpro0wi8.dwo3fsh8.ow4ym5g4.auili1gw.gmql0nx0")
    ActionChains(driver).move_to_element(comment_btn).perform()
    comment_btn.click()
    time.sleep(2)
    
    most_relate_btn=driver.find_element_by_css_selector("div.rq0escxv.l9j0dhe7.du4w35lb.nc684nl6.g0qnabr5")
    ActionChains(driver).move_to_element(most_relate_btn).perform()
    most_relate_btn.click()
    time.sleep(2)
    
    newest_btn=driver.find_element_by_css_selector("div.j34wkznp.qp9yad78.pmk7jnqg.kr520xx4 > div.iqfcb0g7.tojvnm2t.a6sixzi8.k5wvi7nf.q3lfd5jv.pk4s997a.bipmatt0.cebpdrjk.qowsmv63.owwhemhu.dp1hu0rb.dhp61c6y.l9j0dhe7.iyyx5f41.a8s20v7p > div > div > div.rq0escxv.jgsskzai.cwj9ozl2.nwpbqux9.io0zqebd.m5lcvass.fbipl8qg.nwvqtn77.ni8dbmo4.stjgntxs > div > div.j83agx80.cbu4d94t.buofh1pr.l9j0dhe7 > div > div:nth-child(3)")
    ActionChains(driver).move_to_element(newest_btn).perform()
    newest_btn.click()
    time.sleep(2)
    
    flag=0
    
    while flag<2:
        unclick_btn=driver.find_elements_by_css_selector("span.j83agx80.fv0vnmcu.hpfvmrgz")
        if(len(unclick_btn)==1):
            flag+=1
        for btn in unclick_btn:
            try:
                ActionChains(driver).move_to_element(btn).perform()
                btn.click()
                time.sleep(0.1)
        
        #頁面上無更多頁面按鈕
        except StaleElementReferenceException:
            try:
                more_comment_btn=driver.find_element_by_css_selector("span.j83agx80.fv0vnmcu.hpfvmrgz")
                ActionChains(driver).move_to_element(more_comment_btn).perform()
                more_comment_btn.click()
                time.sleep(0.1)
                
            except NoSuchElementException:
                driver.execute_script("window.scrollTo(0,200)")
                print("stop")
                break
            
        except NoSuchElementException:
            driver.execute_script("window.scrollTo(0,200)")
            print("stop")
            break
            
    #Get Comment       
    soup=BeautifulSoup(driver.page_source,"html.parser")
    temp_all_comment_text=[]
    post=soup.find("div",class_="h3gjbzrl qsy8amke")
    all_comment=post.find_all("div",class_="rj1gh0hx buofh1pr ni8dbmo4 stjgntxs hv4rvrfc")
    for comment in all_comment:
        record=[]
        time_label=comment.find("a",class_="oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl m9osqain gpro0wi8 knj5qynh")
        time_text=time_label.find("span",class_="tojvnm2t a6sixzi8 abs2jz4q a8s20v7p t1p8iaqh k5wvi7nf q3lfd5jv pk4s997a bipmatt0 cebpdrjk qowsmv63 owwhemhu dp1hu0rb dhp61c6y iyyx5f41").get_text()
        record.append(time_text)
        record.append(comment.find("span",class_="d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j keod5gw0 nxhoafnm aigsh9s9 d9wwppkn fe6kdd0r mau55g9w c8b282yb mdeji52x e9vueds3 j5wam9gi lrazzd5p oo9gr5id").get_text())
        try:
            content=comment.find("div",class_="kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql").get_text()
        except AttributeError:
            record.append("picture")
        else:
            record.append(content)
            
        temp_all_comment_text.append(record)
    
    now=date.today()
    df=pd.DataFrame(temp_all_comment_text,columns=["Comment_Time","Comment_By","Comment"])
    dest="C:/Users/Jim/Desktop/facebook_自由0518_post_3.csv"
    df.to_csv(dest,encoding='utf_8_sig')

IndentationError: unexpected unindent (<ipython-input-5-de4ed2d69086>, line 48)

In [4]:
#程式內時間處理

from datetime import date
from datetime import datetime
from datetime import timedelta

def time_dealing(current_time,input_string):
    if(input_string.find("小時")>-1):
        
        delta=timedelta(hours=int(input_string[:input_string.find("小時")]))
        output_time=current_time-delta
        output_string=output_time.strftime("%m月%d日 %H:%M:%S")
        
    elif (input_string.find("天")>-1):
        delta=timedelta(days=int(input_string[:input_string.find("天")]))
        output_time=current_time-delta
        output_string=output_time.strftime("%m月%d日 %H:%M:%S")
    else:
        output_string=input_string
    
    return output_string

In [5]:
#字串處理成時間

from datetime import date
from datetime import datetime
from datetime import timedelta

def date_dealing(input_string):
    input_string="2021年"+input_string
    cut_end=input_string.find("日")
    temp=input_string[:cut_end+1]
    output_time=datetime.strptime(temp,("%Y年%m月%d日"))
    
    return output_time

In [6]:
#test area

import pandas as pd

df = pd.read_csv('video_link_by_2021_08_17.csv')
del df['A']
link=df.iloc[0,1]
driver=OpenChromeDriver()
loginfb(driver)

driver.get(link)
time.sleep(2)
driver.execute_script("window.scrollTo(0,400)")

comment_btn=driver.find_element_by_css_selector("div.oajrlxb2.gs1a9yip.g5ia77u1.mtkw9kbi.tlpljxtp.qensuy8j.ppp5ayq2.goun2846.ccm00jje.s44p3ltw.mk2mc5f4.rt8b4zig.n8ej3o3l.agehan2d.sk4xxmp2.rq0escxv.nhd2j8a9.pq6dq46d.mg4g778l.btwxx1t3.pfnyh3mw.p7hjln8o.kvgmc6g5.cxmmr5t8.oygrvhab.hcukyx3x.tgvbjcpo.hpfvmrgz.jb3vyjys.rz4wbd8a.qt6c0cv9.a8nywdso.l9j0dhe7.i1ao9s8h.esuyzwwr.f1sip0of.du4w35lb.lzcic4wl.abiwlrkh.gpro0wi8.dwo3fsh8.ow4ym5g4.auili1gw.gmql0nx0")
ActionChains(driver).move_to_element(comment_btn).perform()
comment_btn.click()
time.sleep(2)

most_relate_btn=driver.find_element_by_css_selector("div.rq0escxv.l9j0dhe7.du4w35lb.nc684nl6.g0qnabr5")
ActionChains(driver).move_to_element(most_relate_btn).perform()
most_relate_btn.click()
time.sleep(2)

newest_btn=driver.find_element_by_css_selector("div.j34wkznp.qp9yad78.pmk7jnqg.kr520xx4 > div.iqfcb0g7.tojvnm2t.a6sixzi8.k5wvi7nf.q3lfd5jv.pk4s997a.bipmatt0.cebpdrjk.qowsmv63.owwhemhu.dp1hu0rb.dhp61c6y.l9j0dhe7.iyyx5f41.a8s20v7p > div > div > div.rq0escxv.jgsskzai.cwj9ozl2.nwpbqux9.io0zqebd.m5lcvass.fbipl8qg.nwvqtn77.ni8dbmo4.stjgntxs > div > div.j83agx80.cbu4d94t.buofh1pr.l9j0dhe7 > div > div:nth-child(3)")
ActionChains(driver).move_to_element(newest_btn).perform()
newest_btn.click()
time.sleep(2)

unclick=driver.find_elements_by_css_selector("span.j83agx80.fv0vnmcu.hpfvmrgz")
ActionChains(driver).move_to_element(unclick[0]).perform()
unclick[0].click()

In [ ]:
https://www.facebook.com/m.ltn.tw/videos/245627427329964
driver=OpenChromeDriver()
loginfb(driver)
getVideoComment("https://www.facebook.com/m.ltn.tw/posts/4287803421305291",driver,"2021/05/18")